In [61]:
import pandas as pd
import numpy as np
import folium
import simplejson,requests
import sys
import time
import import_ipynb
import YouTubeAPI as youtube
import pymysql as db

In [69]:
import konlpy
from konlpy.corpus import kolaw
from konlpy.tag import *
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import copy

In [90]:
a = ['것','가','곯']
for i in a:
    print(type(i))
print((type(a[2])))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [63]:
desc = youtube.description
title = youtube.title
for i in range(len(desc)):
    print('제목 : ' + title[i] + '\n')
    print(desc[i])
    print("---------------------다음---------------------------")

제목 : 🍁1박2일 가을&겨울 국내여행지 추천!! 🚗 전동차로 안동 하회마을 여행하기!! (feat. 뚜벅이) - 여행브이로그(vlog)

#국내여행 #가을여행 #겨울여행 #안동 #하회마을 #뚜벅이 #여행지추천 #1박2일여행지 #단풍 #커플여행 #베낭여행 

📸 Instagram : www.instagram.com/uiland.kr
✉️ Email : uiland118@gmail.com

안녕하세요! "유일랜드" 입니다😊
날씨가 점점 추워지고있어요! 저희는 이번에 안동으로 1박2일 국내여행을 다녀왔어요!
하회마을 - 병산서원 - 낙강물길공원 - 월영교 - 신세동 벽화마을 까지! 영상으로 모두 담아왔습니다😊 이번 영상에서는 저희가 느끼고, 본 하회마을에 대한 정보들을  낱낱이 담았어요✨(2편도 있으니까 기대해주세요!!) 뚜벅이라 쉽지 않은 여행이었지만, 가을엔 단풍과 낙엽을 보기에! 겨울에는 우리나라의 옛 고유의 모습을 느끼기에! 국내여행지로 안동은 너무 이쁘고 좋았던 것 같아요! 자세한 내용은 업로드 된 영상을 참고해주세요! 
오늘도 저희 영상에 함께해 주셔서 고마워요❗️CHEERS 👊

----------------------------------------------------------------------------------

🎵 노래 & 효과음

Epidemic Sound - https://bit.ly/34O1Df7

가입 시 한 달 무료로 사용이 가능합니다:)

----------------------------------------------------------------------------------

🎥 촬영장비

Gopro Hero Black 9
Gopro Hero Black 10
iPhone 12 Pro Max

💻 편집장비

2021 맥북프로 16인치
2021 맥북프로 14인치
---------------------다음---------------------------
제목 : ✨뚜벅이들의 안동 1박2일 국내여행✨인생샷 찍기 좋은 장소

In [64]:
def cleanText(readData): 
# url 제거
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$\-@\.&+:/?=]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',readData)
# 이메일 제거    
    text = re.sub('[-A-Za-z0-9_]+[-A-Za-z0-9_.][@]{1}[-A-Za-z0-9_]+[-A-Za-z0-9_.][.]{1}[A-Za-z]{1,5}',' ',text)
#텍스트에 포함되어 있는 특수 문자 제거    
    text = re.sub('[:=+,#/\?^$.@*\"※~&%ㆍ!』\\‘’|\(\)\[\]\<\>`\'…》✨✔✅❤♡️♥ㅂㅈㄷㄱㅅㅁㄴㅇㄹㅎㅋㅌ\
                  ㅊㅍㅛㅕㅑㅐㅔㅗㅓㅏㅣㅠㅜㅡ]', ' ', text) 
    text = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE).sub(r' ',text)   # 이모티콘 제거
    text = re.sub('\\n\\t\\r',' ',text)                         # 탭, 줄띄우기 등 제거
    return text
print(cleanText(desc[0]))
# 특정 이모티콘 제거 불가 :🌊 🏖 ✉ 등

 국내여행  가을여행  겨울여행  안동  하회마을  뚜벅이  여행지추천  1박2일여행지  단풍  커플여행  베낭여행 

  Instagram   www instagram com uiland kr
✉  Email    

안녕하세요   유일랜드  입니다 
날씨가 점점 추워지고있어요  저희는 이번에 안동으로 1박2일 국내여행을 다녀왔어요 
하회마을 - 병산서원 - 낙강물길공원 - 월영교 - 신세동 벽화마을 까지  영상으로 모두 담아왔습니다  이번 영상에서는 저희가 느끼고  본 하회마을에 대한 정보들을  낱낱이 담았어요  2편도 있으니까 기대해주세요    뚜벅이라 쉽지 않은 여행이었지만  가을엔 단풍과 낙엽을 보기에  겨울에는 우리나라의 옛 고유의 모습을 느끼기에  국내여행지로 안동은 너무 이쁘고 좋았던 것 같아요  자세한 내용은 업로드 된 영상을 참고해주세요  
오늘도 저희 영상에 함께해 주셔서 고마워요❗ CHEERS  

----------------------------------------------------------------------------------

  노래   효과음

Epidemic Sound -  

가입 시 한 달 무료로 사용이 가능합니다  

----------------------------------------------------------------------------------

  촬영장비

Gopro Hero Black 9
Gopro Hero Black 10
iPhone 12 Pro Max

  편집장비

2021 맥북프로 16인치
2021 맥북프로 14인치


In [93]:

n = 1
desc = youtube.description
print(desc[n])
lex_desc = desc[n].split('\n')  # 각 줄마다 split
spl_place = []
for a in lex_desc:                      # 타임라인에 존재하는 장소 추출
    if bool(re.search('^\d{2}:\d{2}',a)):
        sub_place = re.sub('^\d{2}:\d{2} ?','', a)
        spl_place.append(sub_place)
    else:
        continue
print(spl_place)
place = []
for j in spl_place:
    i = j
    if i == "Intro" or i == '인트로' or i == '#인트로':                    # Intro는 제거
        continue
    if '먹방' in i or '여행' in i or '맛집' in i:
        continue
    if "(" in i:
        i = i.split("(")[0]
    if "#" in i:
        i = i.split("#")[-1]  # 뒤에 존재하는 장소 추출
    if "와"in i:
        a = i.split("와")[0] # 와 뒤에 존재하는 장소 추출
        b = i.split("와")[1]
        place.append(a)
        place.append(b)
        continue
    if "/"in i:
        a = i.split("/")[0] # 와 뒤에 존재하는 장소 추출
        b = i.split("/")[1]
        place.append(a)
        place.append(b)
        continue
    place.append(i)
print(place)
if len(place) < 2:
    hannanum = Hannanum()              # Mecab 쓰고 싶은데... 설치를 다시 해도 오류가 뜸..;;
    h_place = hannanum.nouns(cleanText(desc[n]))
    num_place = copy.deepcopy(h_place)
    site = []
    for i in h_place:
        if str.isdigit(i):
            num_place.remove(i)
            continue
        if i == '여행' or i == '여행지':
            continue
        if '여행' in i:
            site.append(i)
            num_place.remove(i)
            continue
        if len(i) == 1:
            num_place.remove(i)
    place = num_place
    print(h_place)
    print(num_place)
    print(site)

#안동 #국내여행지추천 #가을여행 #겨울여행 #뚜벅이 #배낭여행 #병산서원 #낙강물길공원 #월영교 #안동찜닭 #커플여행

📸 Instagram : www.instagram.com/uiland.kr
✉️ Email : uiland118@gmail.com

안녕하세요! 저번주 "안동-하회마을편"에 이어 두번째 안동여행 영상이 드디어 업로드 됐습니다! 😊  둘째날에는 병산서원 - 찜닭거리 - 낙동물길공원 - 월영교! 무려 4곳이나 다녀왔어요 😳 안동은 유명한 관광지 뿐만 아니라 어느 곳도 빼놓을 수 없을 만큼 아름답고 이뻤어요 ✨ 여행을 할때 저희가 뚜벅이라 교통부분에서 어렵고, 불편한 것도 많았지만 뚜벅이 여행하시는 분들을 위해 안동은 엄청 어려운 여행지는 아니었던 것 같아요!! 또, 생각보다 커플분들도 많이 가시더라구요:) 바다의 매력보다 잔잔하고 따뜻한 우리나라의 전통느낌이 그리울때, 또! 나무와 이쁜 꽃들로 힐링이 필요할 땐 안동이 너무 좋은 여행지 인 것 같아요!!  자세한 내용은 업로드 된 영상을 통해 확인해 주세요😊🍁 저희는 더 좋은 영상으로 다음주 이시간 (목요일 저녁 8시)에 다시 만나기로해요💕 오늘도 영상에 함께해주셔서 고마워요🔥


----------------------------------------------------------------------------------

🎵 노래 & 효과음

Epidemic Sound - https://bit.ly/34O1Df7

가입 시 한 달 무료로 사용이 가능합니다:)

----------------------------------------------------------------------------------

🎥 촬영장비

Gopro Hero Black 9
Gopro Hero Black 10
iPhone 12 Pro Max

💻 편집장비

2021 맥북프로 16인치
2021 맥북프로 14인치
[]
[]
['안동', '국내여행지추천', '가을여행', '겨울여행', '뚜벅', '배낭여행', '

In [67]:
local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
con = db.connect(
    host=local,
    user='admin',
    db='Nexpot',
    password='qwertyuiop123',
    charset='utf8'
)
cur = con.cursor()

KeyboardInterrupt: 

In [94]:
place_id = []
place_do =  []
place_si =[]
place_name = []
place_address = []
place_phone = []
place_url = []
place_X = []
place_Y = []
kakao_df = pd.DataFrame(columns=['ID','이름','주소','도', '시',
                                 '전화번호','URL','X좌표','Y좌표'])

url = "https://dapi.kakao.com/v2/local/search/keyword.json?" # kakaomap api에 접근
apikey = "c533ba443ba909af8d277567163d03fe"
for i in range(len(place)):    
    query = str(place[i])
    r = requests.get( url, params = {'query':query}, 
                     headers={'Authorization' : 'KakaoAK ' + apikey } )
    js = simplejson.JSONEncoder().encode(r.json())
    
    # 장소를 입력하여 json파일 형태의 데이터 생성
    rj = r.json()
    # 입력했을 때 결과가 없으면 생략
    if rj['documents'] == []:                        
        continue
    place_id.append(rj['documents'][0]['id'])         # 이후 list에 결과 append
    if rj['meta']['same_name']['selected_region'] == '':
        place_do.append(rj['documents'][0]['address_name'].split()[0])
        place_si.append(rj['documents'][0]['address_name'].split()[1])
    elif len(rj['meta']['same_name']['selected_region'].split()) == 1:
        place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
    else:
        place_do.append(rj['meta']['same_name']['selected_region'].split()[0])
        place_si.append(rj['meta']['same_name']['selected_region'].split()[1])
    place_name.append(rj['documents'][0]['place_name'])
    place_address.append(rj['documents'][0]['address_name'])
    place_phone.append(rj['documents'][0]['phone'])
    place_url.append(rj['documents'][0]['place_url'])
    place_X.append(rj['documents'][0]['x'])
    place_Y.append(rj['documents'][0]['y'])
    
kakao_df=pd.DataFrame([place_id,place_name,place_address,place_do,place_si,place_phone,place_url,place_X,place_Y]).T
kakao_df.columns=['placeID','pname','paddress',
                  'pdo', 'psi','pphone','pdaumURL','pX','pY']
 
kakao_df



,placeID,pname,paddress,pdo,psi,pphone,pdaumURL,pX,pY
0,7999373,안동하회마을,경북 안동시 풍천면 하회리 1176-1,경북,안동시,054-853-0103,http://place.map.kakao.com/7999373,128.518141153901,36.5389828267328
1,821333973,뚜벅게스트하우스,강원 고성군 토성면 봉포리 249-14,강원,고성군,,http://place.map.kakao.com/821333973,128.563696411785,38.2528516162121
2,7892417,병산서원,경북 안동시 풍천면 병산리 31,경북,안동시,054-853-0109,http://place.map.kakao.com/7892417,128.5526253323358,36.540575142988175
3,36022109,낙강물길공원,경북 안동시 상아동 423,경북,안동시,054-840-3433,http://place.map.kakao.com/36022109,128.770944476238,36.5878596847562
4,12819705,월영교,충북 제천시 한수면 송계리,충북,제천시,,http://place.map.kakao.com/12819705,128.082282626708,36.8763315980983
5,1704216412,백선당찜닭 탄방본점,대전 서구 탄방동 721,대전,서구,042-488-5548,http://place.map.kakao.com/1704216412,127.38906715475655,36.34809202222066
6,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울,종로구,02-6954-2910,http://place.map.kakao.com/1779571274,126.983529078926,37.574514077728
7,1793153538,스튜디오두번의봄,경남 창원시 진해구 자은동 138-1,경남,창원시,055-551-1219,http://place.map.kakao.com/1793153538,128.713439663892,35.1553917612933
8,8663308,한국영상대학교,세종특별자치시 장군면 금암리 180-1,세종특별자치시,장군면,044-850-9000,http://place.map.kakao.com/8663308,127.20919616105093,36.46358032130445
9,447978750,업로드PC,울산 울주군 범서읍 사연리 254-2,울산,울주군,,http://place.map.kakao.com/447978750,129.191354997523,35.5762162500048


In [59]:
kakao_df = kakao_df[kakao_df.pdo == '부산']
kakao_df    
'''
['#부산가볼만한곳', '#부산여행', '#부산', '부산', '51번', '일상이여행', '추천', '지역별', '수', '여행지',
 '▶영', '여행지', '1.', '#흰여울해안터널', '2.', '#흰여울문화', '3.', '#태종대', '유원지', '4.', '#영도대교', '영도다리', 
 '5.', '#신기산업카페', '청학배수지전망대', '▶부산', '중심지', '부산진구', '중구', '6.', '#호천마을', '쌈마이웨', '촬영지',
 '7.', '#황령산전망대', '(황령산', '봉수대', '▶광안리', '8.', '#광안리해수욕장', '9.', '#민락항방파제', '▶해운대', '부근', 
 '10', '#마린시티', '영화의거리', '11', '#수영만요트장', '12', '#해운대해수욕장', '13', '#해운대블루라인파크', '14', '#갤러리카페'
 , '#뮤지드블루', '▶서부산', '주변', '15', '#감천문화', '16', '#송도해상케이블카', '17', '#송도해상케이블카', '스카이파크', '18'
 , '#송도용궁구름다', '19', '#다대포생태습지원', '20', '#다대포해수욕장', '21', '#장림포구', '부네치', '▶기장', '주변', '22',
 '#해동용궁사', '23', '#기장카페', '웨이브온', '브랜디드', '촬영', '여행지', '소개', '협찬', '아래', '주소', '문']
'''

"\n['#부산가볼만한곳', '#부산여행', '#부산', '부산', '51번', '일상이여행', '추천', '지역별', '수', '여행지',\n '▶영', '여행지', '1.', '#흰여울해안터널', '2.', '#흰여울문화', '3.', '#태종대', '유원지', '4.', '#영도대교', '영도다리', \n '5.', '#신기산업카페', '청학배수지전망대', '▶부산', '중심지', '부산진구', '중구', '6.', '#호천마을', '쌈마이웨', '촬영지',\n '7.', '#황령산전망대', '(황령산', '봉수대', '▶광안리', '8.', '#광안리해수욕장', '9.', '#민락항방파제', '▶해운대', '부근', \n '10', '#마린시티', '영화의거리', '11', '#수영만요트장', '12', '#해운대해수욕장', '13', '#해운대블루라인파크', '14', '#갤러리카페'\n , '#뮤지드블루', '▶서부산', '주변', '15', '#감천문화', '16', '#송도해상케이블카', '17', '#송도해상케이블카', '스카이파크', '18'\n , '#송도용궁구름다', '19', '#다대포생태습지원', '20', '#다대포해수욕장', '21', '#장림포구', '부네치', '▶기장', '주변', '22',\n '#해동용궁사', '23', '#기장카페', '웨이브온', '브랜디드', '촬영', '여행지', '소개', '협찬', '아래', '주소', '문']\n"

In [56]:
kakao_df=pd.DataFrame([place_id,place_name,place_address,place_do,place_si,place_phone,place_url,place_X,place_Y]).T
kakao_df.columns=['placeID','pname','paddress',
                  'pdo', 'psi','pphone','pdaumURL','pX','pY']
kakao_df                                              # 데이터프레임 생성

,placeID,pname,paddress,pdo,psi,pphone,pdaumURL,pX,pY
0,616634523,제주하늘집,제주특별자치도 제주시 한림읍 옹포리 588-2,제주특별자치도,제주시,064-796-9669,http://place.map.kakao.com/616634523,126.2510181017155,33.402016464349366
1,2083466041,남원큰엉해변,제주특별자치도 서귀포시 남원읍 남원리 2379-3,제주특별자치도,서귀포시,,http://place.map.kakao.com/2083466041,126.703710044663,33.2733501754907
2,82389937,비밀의숲,제주특별자치도 제주시 구좌읍 송당리 2173,제주특별자치도,제주시,,http://place.map.kakao.com/82389937,126.75493209633262,33.45360917744684
3,1263853015,이호테우등대,제주특별자치도 제주시 이호일동,제주특별자치도,제주시,064-712-5223,http://place.map.kakao.com/1263853015,126.451961352582,33.5017949651276
4,2060914594,무지개 해안도로,경남 사천시 용현면 주문리 880-1,경남,사천시,,http://place.map.kakao.com/2060914594,128.038823591836,34.9969640792719
5,317349030,동백포레스트,제주특별자치도 서귀포시 남원읍 신례리 1767,제주특별자치도,서귀포시,0507-1331-2102,http://place.map.kakao.com/317349030,126.63619814081831,33.300308154725585
6,236028555,제주동백수목원,제주특별자치도 서귀포시 남원읍 위미리 927,제주특별자치도,서귀포시,064-764-4473,http://place.map.kakao.com/236028555,126.67799150264848,33.27497497875763
7,213125558,카페글렌코,제주특별자치도 제주시 구좌읍 송당리 2635-8,제주특별자치도,제주시,010-9587-3555,http://place.map.kakao.com/213125558,126.73324152121354,33.43367131314919
8,2046257343,히든클리프호텔&네이쳐 더 인피니티,제주특별자치도 서귀포시 상예동 625,제주특별자치도,서귀포시,064-752-7777,http://place.map.kakao.com/2046257343,126.402651693233,33.254664959304
9,1134184953,애월카페거리,제주특별자치도 제주시 애월읍 애월리,제주특별자치도,제주시,,http://place.map.kakao.com/1134184953,126.309168891527,33.4625871638386


In [57]:
for i in range(len(kakao_df)):   # places 테이블
    idx = f'{i+1}'
    sql = "insert into places values('" + idx + "','" + youtube.video_ids[n] + "','" + \
    kakao_df['placeID'][i] + "','" + kakao_df['pname'][i] + "','" + \
    kakao_df['paddress'][i] +"','" + kakao_df['pdo'][i] + "','" + \
    kakao_df['psi'][i] +"','" + kakao_df['pphone'][i] + "','" + kakao_df['pdaumURL'][i] +\
    "','" + kakao_df['pX'][i]+"','" + kakao_df['pY'][i] + "')"
    try:
        cur.execute(sql)
        con.commit()
    except:
        con.rollback()
        
con.close()